In [1]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v2.4.4 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [2]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.4 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [3]:

#leyendo los csv
df_wc=spark.read.csv("/FileStore/tables/WorldCups.csv",header=True)
df_wcm=spark.read.option("delimiter",";").csv("/FileStore/tables/WorldCupMatches.csv",header=True)
df_wcp=spark.read.csv("/FileStore/tables/WorldCupPlayers.csv",header=True)
#Cargando en temporales
df_wc.createOrReplaceTempView("WorldCup")
df_wcm.createOrReplaceTempView("WorldCupMatches")
df_wcp.createOrReplaceTempView("WorldCupPlayers")
  
#Mostrando la data
spark.sql(""" select x.* from WorldCup x""").show()



+----+------------+----------+--------------+-----------+--------------+-----------+--------------+-------------+----------+
Year| Country| Winner| Runners-Up| Third| Fourth|GoalsScored|QualifiedTeams|MatchesPlayed|Attendance|
+----+------------+----------+--------------+-----------+--------------+-----------+--------------+-------------+----------+
1930| Uruguay| Uruguay| Argentina| USA| Yugoslavia| 70| 13| 18| 590.549|
1934| Italy| Italy|Czechoslovakia| Germany| Austria| 70| 16| 17| 363.000|
1938| France| Italy| Hungary| Brazil| Sweden| 84| 15| 18| 375.700|
1950| Brazil| Uruguay| Brazil| Sweden| Spain| 88| 13| 22| 1.045.246|
1954| Switzerland|Germany FR| Hungary| Austria| Uruguay| 140| 16| 26| 768.607|
1958| Sweden| Brazil| Sweden| France| Germany FR| 126| 16| 35| 819.810|
1962| Chile| Brazil|Czechoslovakia| Chile| Yugoslavia| 89| 16| 32| 893.172|
1966| England| England| Germany FR| Portugal| Soviet Union| 89| 16| 32| 1.563.135|
1970| Mexico| Brazil| Italy| Germany FR| Uruguay| 95| 16| 32| 1.603.975|
1974| Germany|Germany FR| Netherlands| Poland| Brazil| 97| 16| 38| 1.865.753|
1978| Argentina| Argentina| Netherlands| Brazil| Italy| 102| 16| 38| 1.545.791|
1982| Spain| Italy| Germany FR| Poland| France| 146| 24| 52| 2.109.723|
1986| Mexico| Argentina| Germany FR| France| Belgium| 132| 24| 52| 2.394.031|
1990| Italy|Germany FR| Argentina| Italy| England| 115| 24| 52| 2.516.215|
1994| USA| Brazil| Italy| Sweden| Bulgaria| 141| 24| 52| 3.587.538|
1998| France| France| Brazil| Croatia| Netherlands| 171| 32| 64| 2.785.100|
2002| Korea/Japan| Brazil| Germany| Turkey|Korea Republic| 161| 32| 64| 2.705.197|
2006| Germany| Italy| France| Germany| Portugal| 147| 32| 64| 3.359.439|
2010|South Africa| Spain| Netherlands| Germany| Uruguay| 145| 32| 64| 3.178.856|
2014| Brazil| Germany| Argentina|Netherlands| Brazil| 171| 32| 64| 3.386.810|
+----+------------+----------+--------------+-----------+--------------+-----------+--------------+-------------+----------+

In [4]:
#verificando la estructura del dataframe WordlCup
df_wc.printSchema()
print('El dataframe df_wc tiene '+str(df_wc.count())+' registros.')
print('El dataframe df_wc tiene '+str(df_wc.distinct().count())+' registros distintos.')

root
-- Year: string (nullable = true)
-- Country: string (nullable = true)
-- Winner: string (nullable = true)
-- Runners-Up: string (nullable = true)
-- Third: string (nullable = true)
-- Fourth: string (nullable = true)
-- GoalsScored: string (nullable = true)
-- QualifiedTeams: string (nullable = true)
-- MatchesPlayed: string (nullable = true)
-- Attendance: string (nullable = true)

El dataframe df_wc tiene 20 registros.
El dataframe df_wc tiene 20 registros distintos.

In [5]:
#verificando la estructura del dataframe WordlCupPlayers
df_wcp.printSchema()
print('El dataframe df_wcp tiene '+str(df_wcp.count())+' registros.')
print('El dataframe df_wcp tiene '+str(df_wcp.distinct().count())+' registros distintos.')

root
-- RoundID: string (nullable = true)
-- MatchID: string (nullable = true)
-- Team Initials: string (nullable = true)
-- Coach Name: string (nullable = true)
-- Line-up: string (nullable = true)
-- Shirt Number: string (nullable = true)
-- Player Name: string (nullable = true)
-- Position: string (nullable = true)
-- Event: string (nullable = true)

El dataframe df_wcp tiene 37048 registros.
El dataframe df_wcp tiene 37048 registros distintos.

In [6]:
#verificando la estructura del dataframe WordlCupMatches
df_wcm.printSchema()
print('El dataframe df_wc tiene '+str(df_wcm.count())+' registros.')
print('El dataframe df_wc tiene '+str(df_wcm.distinct().count())+' registros distintos.')

root
-- Year: string (nullable = true)
-- Datetime: string (nullable = true)
-- Stage: string (nullable = true)
-- Stadium: string (nullable = true)
-- City: string (nullable = true)
-- HomeTeam Name: string (nullable = true)
-- HomeTeamGoals: string (nullable = true)
-- AwayTeamGoals: string (nullable = true)
-- AwayTeamName: string (nullable = true)
-- Winconditions: string (nullable = true)
-- Attendance: string (nullable = true)
-- HalfTimeHomeGoals: string (nullable = true)
-- HalfTimeAwayGoals: string (nullable = true)
-- Referee: string (nullable = true)
-- Assistant1: string (nullable = true)
-- Assistant2: string (nullable = true)
-- RoundID: string (nullable = true)
-- MatchID: string (nullable = true)
-- HomeTeamInitials: string (nullable = true)
-- AwayTeamInitials: string (nullable = true)

El dataframe df_wc tiene 836 registros.
El dataframe df_wc tiene 836 registros distintos.

In [7]:
from pyspark.sql.functions import desc, asc, col, column, expr, instr,length, substring, regexp_replace,trim,lit,initcap, sum,concat
#
#Transformaciones al datraframe WordlCupPlayers 
# El campo Event guarda un dato del tipo "G43' G87'", lo que significa que el jugador usar dos goles, por lo que necesitamos contar el número de "G"
# De igual manera podemos obtener el número de penales y tarjetas
df_wcp1=df_wcp.withColumn('POSICION_JUGADOR',expr("case when position='C' THEN  'Captain' WHEN position='GK' THEN 'Goalkeeper' ELSE 'Other' end "))\
             .withColumn('NOMBRE_JUGADOR', initcap(regexp_replace('Player Name','�','u')))\
             .withColumn('NUMERO_GOLES',length('Event')-length(trim(regexp_replace('Event','G',''))))\
             .withColumn('NUMERO_PENALES',length('Event')-length(trim(regexp_replace('Event','P',''))))\
             .withColumn('NUMERO_PENALES_FALLADOS',length('Event')-length(trim(regexp_replace('Event','MP',''))))\
             .withColumn('NUMERO_TARJETAS_ROJAS',length('Event')-length(trim(regexp_replace('Event','R',''))))
#Reemplazo los valores nulos con 0                 
df_wcp1=df_wcp1.withColumn('NUMERO_GOLES',expr("case when NUMERO_GOLES is null then 0 else NUMERO_GOLES end "))\
              .withColumn('NUMERO_PENALES',expr("case when NUMERO_PENALES is null then 0 else NUMERO_PENALES end "))\
              .withColumn('NUMERO_PENALES_FALLADOS',expr("case when NUMERO_PENALES_FALLADOS is null then 0 else NUMERO_PENALES_FALLADOS end "))\
              .withColumn('NUMERO_TARJETAS_ROJAS',expr("case when NUMERO_TARJETAS_ROJAS is null then 0 else NUMERO_TARJETAS_ROJAS end "))\
              .withColumnRenamed('Team Initials','INICIALES_PAIS')\
              .drop('Player Name','Position','Shirt Number')

#Sumarizo  por nombre de jugador , posición e iniciales de país
df_wcp_rep=df_wcp1.select('NOMBRE_JUGADOR','POSICION_JUGADOR','INICIALES_PAIS','NUMERO_GOLES','NUMERO_PENALES','NUMERO_PENALES_FALLADOS','NUMERO_TARJETAS_ROJAS')\
.groupby('NOMBRE_JUGADOR','POSICION_JUGADOR','INICIALES_PAIS')\
.sum('NUMERO_GOLES','NUMERO_PENALES','NUMERO_PENALES_FALLADOS','NUMERO_TARJETAS_ROJAS')

#Renombro columnas
df_wcp_rep=df_wcp_rep.withColumnRenamed('sum(NUMERO_PENALES)','NUMERO_PENALES')\
                     .withColumnRenamed('sum(NUMERO_PENALES_FALLADOS)','NUMERO_PENALES_FALLADOS')\
                     .withColumnRenamed('sum(NUMERO_GOLES)','NUMERO_GOLES')\
                     .withColumnRenamed('sum(NUMERO_TARJETAS_ROJAS)','NUMERO_TARJETAS_ROJAS')

df_wcp_rep_final=df_wcp_rep.orderBy(desc('NUMERO_GOLES'))
df_wcp_rep_final.show()


+--------------------+----------------+--------------+------------+--------------+-----------------------+---------------------+
 NOMBRE_JUGADOR|POSICION_JUGADOR|INICIALES_PAIS|NUMERO_GOLES|NUMERO_PENALES|NUMERO_PENALES_FALLADOS|NUMERO_TARJETAS_ROJAS|
+--------------------+----------------+--------------+------------+--------------+-----------------------+---------------------+
 Klose| Other| GER| 16| 0| 0| 1|
 Ronaldo| Other| BRA| 14| 1| 0| 0|
 Gerd Mueller| Other| FRG| 13| 1| 0| 0|
 Just Fontaine| Other| FRA| 13| 0| 0| 0|
Pelu (edson Arant...| Other| BRA| 12| 0| 0| 0|
 Sandor Kocsis| Other| HUN| 11| 0| 0| 0|
 Helmut Rahn| Other| FRG| 10| 0| 0| 0|
 Grzegorz Lato| Other| POL| 10| 0| 0| 0|
 Muller| Other| GER| 10| 1| 0| 0|
 Vava| Other| BRA| 9| 0| 0| 0|
 Paolo Rossi| Other| ITA| 9| 0| 0| 0|
 Jairzinho| Other| BRA| 9| 0| 0| 0|
 Teofilo Cubillas| Other| PER| 8| 2| 0| 0|
 Guillermo Stabile| Other| ARG| 8| 0| 0| 0|
 Gary Lineker| Other| ENG| 8| 2| 0| 0|
 Ademir| Other| BRA| 8| 0| 0| 0|
 Oscar Miguez| Other| URU| 8| 0| 0| 0|
 David Villa| Other| ESP| 7| 3| 2| 0|
 Rivaldo| Other| BRA| 7| 1| 0| 0|
 Andrzej Szarmach| Other| POL| 7| 0| 0| 0|
+--------------------+----------------+--------------+------------+--------------+-----------------------+---------------------+
only showing top 20 rows

In [8]:
#guardando en el filestore
df_wcp_rep_final.write.format("csv").mode("overwrite").option("path","/FileStore/tables/reporte_estadistica_jugadores/").option("header","true").save()


In [9]:
#recuperando el archivo guardado
df_wcp_rep_csv=spark.read.csv("/FileStore/tables/reporte_estadistica_jugadores/",header=True)
df_wcp_rep_csv.show()

+--------------------+----------------+--------------+------------+--------------+-----------------------+---------------------+
 NOMBRE_JUGADOR|POSICION_JUGADOR|INICIALES_PAIS|NUMERO_GOLES|NUMERO_PENALES|NUMERO_PENALES_FALLADOS|NUMERO_TARJETAS_ROJAS|
+--------------------+----------------+--------------+------------+--------------+-----------------------+---------------------+
Lizardo Nue Rodri...| Other| PER| 0| 0| 0| 0|
 Hany Kamel| Other| EGY| 0| 0| 0| 0|
 Knut Brynildsen| Other| NOR| 0| 0| 0| 0|
 Juan Gomez| Other| MEX| 0| 0| 0| 0|
 Joseph Mermans| Captain| BEL| 0| 0| 0| 0|
 Jeff Van Der Linden| Other| BEL| 0| 0| 0| 0|
 Manuel Astorga| Other| CHI| 0| 0| 0| 0|
 Leif Malberg| Other| SWE| 0| 0| 0| 0|
 Jan Pivarnik| Other| TCH| 0| 0| 0| 0|
 Ivano Bordon| Other| ITA| 0| 0| 0| 0|
 Wilhelm Kreuz| Other| AUT| 0| 0| 0| 0|
 Heinrich Strasser| Other| AUT| 0| 0| 0| 0|
 Miguel Gutierrez| Other| PER| 0| 0| 0| 0|
 Thomas Finney| Other| NIR| 0| 0| 0| 0|
 Tino Lettieri| Other| CAN| 0| 0| 0| 0|
 Mircea Rednic| Other| ROU| 0| 0| 0| 0|
 Robert Jarni| Other| YUG| 0| 0| 0| 0|
 Mo Johnston| Other| SCO| 0| 1| 0| 0|
 Miguel Davis| Other| CRC| 0| 0| 0| 0|
 Emilio Butragueno| Captain| ESP| 0| 0| 0| 0|
+--------------------+----------------+--------------+------------+--------------+-----------------------+---------------------+
only showing top 20 rows

In [10]:
#mostrándolo en formato Pandas
df_pandas_wcp_csv=df_wcp_rep_csv.toPandas()
df_pandas_wcp_csv.sort_values(by=['NUMERO_GOLES'],inplace=True, ascending=False)
df_pandas_wcp_csv.head()



,NOMBRE_JUGADOR,POSICION_JUGADOR,INICIALES_PAIS,NUMERO_GOLES,NUMERO_PENALES,NUMERO_PENALES_FALLADOS,NUMERO_TARJETAS_ROJAS
8376,Vava,Other,BRA,9,0,0,0
8377,Paolo Rossi,Other,ITA,9,0,0,0
8378,Jairzinho,Other,BRA,9,0,0,0
8379,Ademir,Other,BRA,8,0,0,0
8380,Guillermo Stabile,Other,ARG,8,0,0,0


In [11]:
#Verificando para un jugador
df_pandas_wcp_csv_f=df_pandas_wcp_csv[df_pandas_wcp_csv.NOMBRE_JUGADOR == 'Klose']
df_pandas_wcp_csv_f.head()

,NOMBRE_JUGADOR,POSICION_JUGADOR,INICIALES_PAIS,NUMERO_GOLES,NUMERO_PENALES,NUMERO_PENALES_FALLADOS,NUMERO_TARJETAS_ROJAS
8367,Klose,Other,GER,16,0,0,1


In [12]:
#Realizando transformaciones para el segundo reporte.
df_wcm1=df_wcm.withColumn('FECHA_PARTIDO',substring('Datetime',1,12))\
             .withColumn('NOMBRE_REFEREE', initcap(regexp_replace('Referee','�','u')))\
             .withColumn('ESTADIO', initcap('Stadium'))
df_wcm1=df_wcm1.select('RoundID','MatchID' , 'FECHA_PARTIDO','NOMBRE_REFEREE','ESTADIO','Year',
                      col('HomeTeam Name').alias('EQUIPO DE CASA'),col('AwayTeamName').alias('EQUIPO EXTERNO'))
df_wc1=df_wc.withColumn('MUNDIAL', concat('Country',lit('-'),'Year'))
df_wc1=df_wc1.withColumnRenamed('Year','ANIO')
df_wc1=df_wc1.select('MUNDIAL','ANIO')
#filtrando sólo PERU 
df_wcp1=df_wcp1.filter(col('INICIALES_PAIS')=='PER')





In [13]:
#revisando el dataframe de worldcupmatches formateado
df_wcm1.show()

+-------+-------+-------------+--------------------+--------------------+----+--------------+--------------+
RoundID|MatchID|FECHA_PARTIDO| NOMBRE_REFEREE| ESTADIO|Year|EQUIPO DE CASA|EQUIPO EXTERNO|
+-------+-------+-------------+--------------------+--------------------+----+--------------+--------------+
 201| 1096| 13 Jul 1930 |Lombardi Domingo ...| Pocitos|1930| France| Mexico|
 201| 1090| 13 Jul 1930 | Macias Jose (arg)| Parque Central|1930| USA| Belgium|
 201| 1093| 14 Jul 1930 | Tejada Anibal (uru)| Parque Central|1930| Yugoslavia| Brazil|
 201| 1098| 14 Jul 1930 |Warnken Alberto (...| Pocitos|1930| Romania| Peru|
 201| 1085| 15 Jul 1930 | Rego Gilberto (bra)| Parque Central|1930| Argentina| France|
 201| 1095| 16 Jul 1930 |Cristophe Henry (...| Parque Central|1930| Chile| Mexico|
 201| 1092| 17 Jul 1930 |Mateucci Francisc...| Parque Central|1930| Yugoslavia| Bolivia|
 201| 1097| 17 Jul 1930 | Macias Jose (arg)| Parque Central|1930| USA| Paraguay|
 201| 1099| 18 Jul 1930 | Langenus Jean (bel)| Estadio Centenario|1930| Uruguay| Peru|
 201| 1094| 19 Jul 1930 | Tejada Anibal (uru)| Estadio Centenario|1930| Chile| France|
 201| 1086| 19 Jul 1930 |Saucedo Ulises (bol)| Estadio Centenario|1930| Argentina| Mexico|
 201| 1091| 20 Jul 1930 | Balway Thomas (fra)| Estadio Centenario|1930| Brazil| Bolivia|
 201| 1089| 20 Jul 1930 |Vallarino Ricardo...| Estadio Centenario|1930| Paraguay| Belgium|
 201| 1100| 21 Jul 1930 | Rego Gilberto (bra)| Estadio Centenario|1930| Uruguay| Romania|
 201| 1084| 22 Jul 1930 | Langenus Jean (bel)| Estadio Centenario|1930| Argentina| Chile|
 202| 1088| 26 Jul 1930 | Langenus Jean (bel)| Estadio Centenario|1930| Argentina| USA|
 202| 1101| 27 Jul 1930 | Rego Gilberto (bra)| Estadio Centenario|1930| Uruguay| Yugoslavia|
 405| 1087| 30 Jul 1930 | Langenus Jean (bel)| Estadio Centenario|1930| Uruguay| Argentina|
 204| 1104| 27 May 1934 |Van Moorsel Johan...|Stadio Benito Mus...|1934| Austria| France|
 204| 1119| 27 May 1934 |Barlassina Rinald...| Giorgio Ascarelli|1934| Hungary| Egypt|
+-------+-------+-------------+--------------------+--------------------+----+--------------+--------------+
only showing top 20 rows

In [14]:
#revisando el dataframe de worldcup formateado
df_wc1.show()

+-----------------+----+
 MUNDIAL|ANIO|
+-----------------+----+
 Uruguay-1930|1930|
 Italy-1934|1934|
 France-1938|1938|
 Brazil-1950|1950|
 Switzerland-1954|1954|
 Sweden-1958|1958|
 Chile-1962|1962|
 England-1966|1966|
 Mexico-1970|1970|
 Germany-1974|1974|
 Argentina-1978|1978|
 Spain-1982|1982|
 Mexico-1986|1986|
 Italy-1990|1990|
 USA-1994|1994|
 France-1998|1998|
 Korea/Japan-2002|2002|
 Germany-2006|2006|
South Africa-2010|2010|
 Brazil-2014|2014|
+-----------------+----+

In [15]:
#revisando el dataframe de worldcuplayer formateado
df_wcp1.show()

+-------+-------+--------------+-------------------+-------+-----+----------------+--------------------+------------+--------------+-----------------------+---------------------+
RoundID|MatchID|INICIALES_PAIS| Coach Name|Line-up|Event|POSICION_JUGADOR| NOMBRE_JUGADOR|NUMERO_GOLES|NUMERO_PENALES|NUMERO_PENALES_FALLADOS|NUMERO_TARJETAS_ROJAS|
+-------+-------+--------------+-------------------+-------+-----+----------------+--------------------+------------+--------------+-----------------------+---------------------+
 201| 1098| PER|BRU Francisco (ESP)| S| null| Goalkeeper| Juan Valdivieso| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| S| null| Other|Alejandro Villanueva| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| S| null| Other| Julio Lores| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| S| R70'| Captain| Placido Galindo| 0| 0| 0| 1|
 201| 1098| PER|BRU Francisco (ESP)| S| null| Other| Domingo Garcia| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| S| null| Other| Jose Maria Lavalle| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| S| null| Other| Mario De Las Casas| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| S| null| Other| Alberto Denegri| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| S| null| Other| Demetrio Neyra| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| S| null| Other| Alberto Soria| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| S| G75'| Other| Luis Souza| 1| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| N| null| Other| Jorge Pardon| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| N| null| Other| Jose Flores| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| N| null| Other| Antonio Maquilon| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| N| null| Other|Lizardo Nue Rodri...| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| N| null| Other| Julio Quintana| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| N| null| Other| Arturo Fernandez| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| N| null| Other| Pablo Pacheco| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| N| null| Other| Carlos Cilloniz| 0| 0| 0| 0|
 201| 1098| PER|BRU Francisco (ESP)| N| null| Other| Jorge Gongora| 0| 0| 0| 0|
+-------+-------+--------------+-------------------+-------+-----+----------------+--------------------+------------+--------------+-----------------------+---------------------+
only showing top 20 rows

In [16]:
#join del dataframe de worldcup con el de worldcupmatch
df_wc_join_wcm=df_wc1.join(df_wcm1,df_wc1.ANIO==df_wcm1.Year,'inner')
#viendo el resultado del primer join
df_wc_join_wcm.show()


+------------+----+-------+-------+-------------+--------------------+--------------------+----+--------------+--------------+
 MUNDIAL|ANIO|RoundID|MatchID|FECHA_PARTIDO| NOMBRE_REFEREE| ESTADIO|Year|EQUIPO DE CASA|EQUIPO EXTERNO|
+------------+----+-------+-------+-------------+--------------------+--------------------+----+--------------+--------------+
Uruguay-1930|1930| 201| 1096| 13 Jul 1930 |Lombardi Domingo ...| Pocitos|1930| France| Mexico|
Uruguay-1930|1930| 201| 1090| 13 Jul 1930 | Macias Jose (arg)| Parque Central|1930| USA| Belgium|
Uruguay-1930|1930| 201| 1093| 14 Jul 1930 | Tejada Anibal (uru)| Parque Central|1930| Yugoslavia| Brazil|
Uruguay-1930|1930| 201| 1098| 14 Jul 1930 |Warnken Alberto (...| Pocitos|1930| Romania| Peru|
Uruguay-1930|1930| 201| 1085| 15 Jul 1930 | Rego Gilberto (bra)| Parque Central|1930| Argentina| France|
Uruguay-1930|1930| 201| 1095| 16 Jul 1930 |Cristophe Henry (...| Parque Central|1930| Chile| Mexico|
Uruguay-1930|1930| 201| 1092| 17 Jul 1930 |Mateucci Francisc...| Parque Central|1930| Yugoslavia| Bolivia|
Uruguay-1930|1930| 201| 1097| 17 Jul 1930 | Macias Jose (arg)| Parque Central|1930| USA| Paraguay|
Uruguay-1930|1930| 201| 1099| 18 Jul 1930 | Langenus Jean (bel)| Estadio Centenario|1930| Uruguay| Peru|
Uruguay-1930|1930| 201| 1094| 19 Jul 1930 | Tejada Anibal (uru)| Estadio Centenario|1930| Chile| France|
Uruguay-1930|1930| 201| 1086| 19 Jul 1930 |Saucedo Ulises (bol)| Estadio Centenario|1930| Argentina| Mexico|
Uruguay-1930|1930| 201| 1091| 20 Jul 1930 | Balway Thomas (fra)| Estadio Centenario|1930| Brazil| Bolivia|
Uruguay-1930|1930| 201| 1089| 20 Jul 1930 |Vallarino Ricardo...| Estadio Centenario|1930| Paraguay| Belgium|
Uruguay-1930|1930| 201| 1100| 21 Jul 1930 | Rego Gilberto (bra)| Estadio Centenario|1930| Uruguay| Romania|
Uruguay-1930|1930| 201| 1084| 22 Jul 1930 | Langenus Jean (bel)| Estadio Centenario|1930| Argentina| Chile|
Uruguay-1930|1930| 202| 1088| 26 Jul 1930 | Langenus Jean (bel)| Estadio Centenario|1930| Argentina| USA|
Uruguay-1930|1930| 202| 1101| 27 Jul 1930 | Rego Gilberto (bra)| Estadio Centenario|1930| Uruguay| Yugoslavia|
Uruguay-1930|1930| 405| 1087| 30 Jul 1930 | Langenus Jean (bel)| Estadio Centenario|1930| Uruguay| Argentina|
 Italy-1934|1934| 204| 1104| 27 May 1934 |Van Moorsel Johan...|Stadio Benito Mus...|1934| Austria| France|
 Italy-1934|1934| 204| 1119| 27 May 1934 |Barlassina Rinald...| Giorgio Ascarelli|1934| Hungary| Egypt|
+------------+----+-------+-------+-------------+--------------------+--------------------+----+--------------+--------------+
only showing top 20 rows

In [17]:
#uniendo el tercer dataframe(worldcupplayer) al join anterior
df_wc_join_wcm_join_wcp=df_wcp1.join(df_wc_join_wcm,df_wc_join_wcm.MatchID==df_wcp1.MatchID,'inner')
df_mundiales_peruanos_rep=df_wc_join_wcm_join_wcp.select('MUNDIAL','FECHA_PARTIDO','EQUIPO DE CASA',
                               'EQUIPO EXTERNO','NOMBRE_JUGADOR','POSICION_JUGADOR','INICIALES_PAIS','NOMBRE_REFEREE').orderBy('ANIO')
df_mundiales_peruanos_rep.show()

+------------+-------------+--------------+--------------+--------------------+----------------+--------------+--------------------+
 MUNDIAL|FECHA_PARTIDO|EQUIPO DE CASA|EQUIPO EXTERNO| NOMBRE_JUGADOR|POSICION_JUGADOR|INICIALES_PAIS| NOMBRE_REFEREE|
+------------+-------------+--------------+--------------+--------------------+----------------+--------------+--------------------+
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Juan Valdivieso| Goalkeeper| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru|Alejandro Villanueva| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Julio Lores| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Placido Galindo| Captain| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Domingo Garcia| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Jose Maria Lavalle| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Mario De Las Casas| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Alberto Denegri| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Demetrio Neyra| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Alberto Soria| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Luis Souza| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Jorge Pardon| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Jose Flores| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Antonio Maquilon| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru|Lizardo Nue Rodri...| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Julio Quintana| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Arturo Fernandez| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Pablo Pacheco| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Carlos Cilloniz| Other| PER|Warnken Alberto (...|
Uruguay-1930| 14 Jul 1930 | Romania| Peru| Jorge Gongora| Other| PER|Warnken Alberto (...|
+------------+-------------+--------------+--------------+--------------------+----------------+--------------+--------------------+
only showing top 20 rows

In [18]:
#guardando el segundo reporte de mundiales peruanos
df_mundiales_peruanos_rep.write.format("csv").mode("overwrite").option("path","/FileStore/tables/reporte_mundiales_peruanos/").option("header","true").save()
#recuperandolo en csv
df_mundiales_peruanos_rep_csv=spark.read.csv("/FileStore/tables/reporte_mundiales_peruanos/",header=True)
df_mundiales_peruanos_rep_csv.show()



+--------------+-------------+--------------+--------------+-------------------+----------------+--------------+------------------+
 MUNDIAL|FECHA_PARTIDO|EQUIPO DE CASA|EQUIPO EXTERNO| NOMBRE_JUGADOR|POSICION_JUGADOR|INICIALES_PAIS| NOMBRE_REFEREE|
+--------------+-------------+--------------+--------------+-------------------+----------------+--------------+------------------+
Argentina-1978| 03 Jun 1978| Peru| Scotland| Ramon Quiroga| Goalkeeper| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Jaime Duarte| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Rodolfo Manzo| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Hector Chumpitaz| Captain| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Ruben Diaz| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Jose Velasquez| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Juan Munante| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Cesar Cueto| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Teofilo Cubillas| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland|Juan Carlos Oblitas| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Guillermo La Rosa| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Ottorino Sartor| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Percy Rojas| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Roberto Mosquera| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Juan Caceres| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Jose Navarro| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| German Leguia| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Raul Gorriti| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Alfredo Quesada| Other| PER|Eriksson Ulf (swe)|
Argentina-1978| 03 Jun 1978| Peru| Scotland| Ernesto Labarthe| Other| PER|Eriksson Ulf (swe)|
+--------------+-------------+--------------+--------------+-------------------+----------------+--------------+------------------+
only showing top 20 rows

In [19]:
#mostrándolo en formato panda
df_pandas_mundiales_peruanos=df_mundiales_peruanos_rep_csv.toPandas()
df_pandas_mundiales_peruanos.head()

,MUNDIAL,FECHA_PARTIDO,EQUIPO DE CASA,EQUIPO EXTERNO,NOMBRE_JUGADOR,POSICION_JUGADOR,INICIALES_PAIS,NOMBRE_REFEREE
0,Argentina-1978,03 Jun 1978,Peru,Scotland,Ramon Quiroga,Goalkeeper,PER,Eriksson Ulf (swe)
1,Argentina-1978,03 Jun 1978,Peru,Scotland,Jaime Duarte,Other,PER,Eriksson Ulf (swe)
2,Argentina-1978,03 Jun 1978,Peru,Scotland,Rodolfo Manzo,Other,PER,Eriksson Ulf (swe)
3,Argentina-1978,03 Jun 1978,Peru,Scotland,Hector Chumpitaz,Captain,PER,Eriksson Ulf (swe)
4,Argentina-1978,03 Jun 1978,Peru,Scotland,Ruben Diaz,Other,PER,Eriksson Ulf (swe)
